# Computing the Partion Function and Related Thermodynamic Quantities

The schrodinger.X() modules provide eigenstates that can be used directly in an ensemble of your choice (each with there own unique free energy). In this document, we will cover the availible ensembles implimented in QWavE and how to do some simple boltzmann statistics and evaluation of relevant thermodynamic quantities.

## Canonical Ensemble

The canonical ensemble represents the possible states of a mechanical system in thermal equilibrium with a heat bath at a fixed temperature. The principle thermodynamic variables of the canonical ensemble are the temperature ($T$), number of particles ($N$), and volume of the system ($V$). Making this an $NVT$ system (more on this in another exercise). The partition function within the canonical ensemble is computed as:

$$ q(T) = \sum_j \text{exp}(\frac{-e_{j}}{k_{b}T}) $$

and the probability of accessing an eigenstate at themperate $T$:

$$ p(j,T) = \frac{\text{exp}(\frac{-e_{j}}{k_{b}T})}{q(t)} $$

where $e_{j}$ are the eigen energies from the Schrodinger equation, and $k_{b}$ is the Boltzmann constant. Once you have evaluated the eigenvalues, you simply need to supply them to the canonical_q() module (using appropriate units).

In [ ]:
# Load required modules
from QWavE.qwave import schrodinger, partition_func
import numpy as np
from scipy import constants
import matplotlib.pyplot as plt

# import some constants
h = constants.h/(2*np.pi) # planck constant
aum = constants.physical_constants['atomic unit of mass'][0] # mass of electron
proton_mass = constants.physical_constants['atomic mass constant'][0] # mass of a proton
hartree_j = constants.physical_constants['Hartree energy'][0] # convert hartree to J
bohr_m = constants.physical_constants['Bohr radius'][0] # convert from bohr to meters

Lets evaluate the canonical partition function for a two-state system. Lets put the H atom in a 1D box and get the first two eigenstates of the system.

In [ ]:
# We will use the example of a particle in an arbitrary potential (one_d_potential.csv)
# This potential has the functional form ""

# Evaluate 1-D Schnrodinger Equation

bl = 10.0    # bohr
mass = 32 * proton_mass/aum # mass of a O2 in atomic units
pot = './one_d_potential.csv' # potential model. Current options are 'piab', 'para', or path to csv file
             # piab --> V(x) = 0
             # para --> V(x) = 1/2 * x**2
             # file/to/csv --> comma separated file as described in one_d_potential.csv

E_1D, wave, pot = schrodinger.schrsol_1d(mass,bl,pot)

In [ ]:
# Evaluate the canonical partition function

temp = np.linspace(5,5000,500) # temperature range to evaluate partition function (Kelvin)
Q,P = partition_func.canonical_q(E_1D,temp) # evaluate canonical partition function

canonical_q() returns the canoncial partition function over the desired temperature range (**Q**) and the probablities of finding each eigen state (**P**). $e_j$ from the schrodinger equation solver are in units of Hartree and the temperature range is provided in Kelvin. 

The partion function itself isn't particularly useful, but it is useful in obtaining some Boltzamnn statistics of your system. We can now plot the probalities of accessing each eigen state over the specified temperature range. In the next module, we will cover how to use **Q** and **P** to evaluate useful thermodynamic quantities.

Notes:
1. There are known numerical issues with the canonical_q() function at low temperatures (typically observed in systems where the mass of the particle is less than or equal to the mass of a Hydrogen atom (~1800 amu). If you are dealing with such cases, the partition function at these low temperatures will be set to `0` and the probabilities are also set such that the probability of finding the first eigenstate is `1` and all other eigen states are 0. It is recommended to choose a temperature window that does not cause such errors. 
2. The parition function (and probabilities) are only as reliable as the number of eigen states that you include in your calculation. By default, QWavE will only report 10 eigen values. Should you need more, rerun the schrodinger equation solver and increase `eig_len` appropriately.
3. When evaluating the total partition function using the 1D or 2D schrsol_X() modules, you are inherently making an assumption that the total partition function is seperable (i.e. for the 1D solver: $q^{total} = q^{x}_{1D}q^{y}_{1D}q^{z}_{1D}$). In order to get "Total" thermodynamic quantities, one would need to use the product of individual partition functions. For the purposes of this demonstration, we will be using the 1D partition functions.

In [ ]:
# Probabilities of occupying a state at temperature T

# Plot only the first 4 eigenstates
P0 = P[:,0]
P1 = P[:,1]
P2 = P[:,2]
P3 = P[:,3]

plt.plot(temp,P0,color='blue',label='E0')
plt.plot(temp,P1,color='red',label='E1')
plt.plot(temp,P2,color='green',label='E2')
plt.plot(temp,P3,color='orange',label='E3')
plt.xlabel('Temperature (K)',size=14)
plt.ylabel('Probability',size=14)
plt.xticks(size=14)
plt.yticks(size=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

print("""As you can see, the ground state is populated at low temperatures and the excited states are empty.
As the temperature increases the probability of occupying excited states increases.
As T approaches infinity, the probabilitly of being in any state approaches sum_{i=0}^{2} e_{i}/2 """)

# and the sum of all P should equal 1 at any temperature
Total_P = []
P_trans = P.T
for t in range(len(temp)):
    Total_P.append(np.sum(P_trans[:,t]))
    
plt.plot(temp,Total_P,color='black')
plt.xlabel('Temperature (K)',size=14)
plt.ylabel('Total Prob',size=14)
plt.xticks(size=14)
plt.yticks(size=14)
plt.tight_layout()
plt.show()

## Canonical Ensemble Thermodynamics

### Average Energy (Internal Energy)

Within the Canonical Ensemble, our natural variables are $N, T$, and $V$, which are fixed at desired values. The 

Another useful quantity is the average energy ($<E>$) or internal energy (U). The average energy (or ensemble average) within the canonical ensemble is defined as:

$$ U = <E> = \sum_{j} e_{j}p_{j} = \frac{\sum_{j} e_{j} \text{exp}(-e_{j}/k_{b}T)}{\sum_{j} \text{exp}(-e_{j}/k_{b}T)} $$

(which is how QWavE evaluates the average energy and internal energy utilizing the probabilites as described in the first equation)

or by differentiation:

$$ <E> = -\frac{\partial \text{ln}(q(T))}{\partial \beta} $$

where $\beta$ is $1/k_{b}T$. The variance in the energy can also be defined as:

In [ ]:
from QWavE.qwave import thermo

U = thermo.internal_U(E_1D,P)

plt.plot(temp,U,linewidth=5,color='blue')
plt.ylabel(r'1D Internal Energy (Hartree)',size=14)
plt.xlabel(r'Temperature (K)',size=14)
plt.xticks(size=14)
plt.yticks(size=14)
plt.show()

### Helmholtz Free Energy and Entropy

The Helmholtz free eneergy and entropy are straight forward to compute. Once you have the partition function the Helmholtz free energy can be computes as:  

$$A = -k_{b}T\ln(\prod_{i=\text{DOF}}q_{i})$$

where $\ln(\prod_{i=\text{DOF}}q_{i})$ is the product of all the seperable degrees of freedom (DOF). In the case of the 1D Helmholtz free energy, the form above reduces to:

$$A = -k_{b}T\ln(q)$$

In [ ]:
A = thermo.helmholtz_F(Q,temp)

plt.plot(temp,A,linewidth=5,color='red')
plt.ylabel(r'1D Free Energy (Hartree)',size=14)
plt.xlabel(r'Temperature (K)',size=14)
plt.xticks(size=14)
plt.yticks(size=14)
plt.show()

The 1D entropy can be evaluated using the probabilities:

$$ S = -p_{j}*\ln(p_{j})$$

or by differentiating the Helmholtz free energy by Temperature:

$$ S = \frac{-\partial A}{\partial T} $$

Examples of both are demonstrated below

In [ ]:
S_prob = thermo.entropy_S(P)
S_diff = np.diff(-A)/np.diff(temp) # you will loose one data point with this method (last data point)

plt.plot(temp,S_prob,lw=5,label='S from prob')
plt.plot(temp[0:-1],S_diff,ls='dotted',lw=5,label='S from diff')
plt.ylabel(r'1D Entropy (Hartree/K)',size=14)
plt.xlabel(r'Temperature (K)',size=14)
plt.xticks(size=14)
plt.yticks(size=14)
plt.legend(fontsize=14)
plt.show()

## Predefined Partition Functions (returning soon)

For ease of use, we have also incorporated other commonly used partition functions dervied from the canonical partition function. These include the: harmonic oscillator, hindered translator, rigid rotor, and others. We will show example of using these other functions in another jupyter notebook. In this example, we will harmonic oscillator partition function to find the average energy and heat capacity of an einstein crystal.

The harmonic oscillator partition function is defined as:

$$ q_{HO}(T) = \frac{\exp{(\frac{\nu}{2 k_{b}T})}}{1-\exp{(\frac{\nu}{k_{b}T})}}$$

where $\nu$ is a frequency (cm$^{-1}$)

As you can see, both modules give the same result. NOTE: in order to achieve perfect parity, the box length needs to be adjusted to "match" with the curvature of the potential, change the box length from 2 to 10 to see what happens. As such, it is highly recommended to use the q_HO (or other analytic expressions) when you know the shape of the potential.

Now, lets run through the same exercise to get the average energy and Cv of the einstein crystal

## Grand Canonical Partition function ???

## Microcanonical Partition function ???

Which again, is as expected. 


Hopefully this introductory notebook has familiarized you with the selection of partition functions as well as some basic statistics one can do in QWavE. If there are any bugs, issues or comments, please direct them to the authors at on the GitHub repository at https://github.com/cwaitt/QWavE.